# 01-Kaggle House Prices 2023

## Import data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/House_Price_dataset.csv')

## Data structure

In [3]:
df.shape

(168446, 20)

In [4]:
df.head()

,property_id,location_id,page_url,property_type,price,location,city,province_name,latitude,longitude,baths,area,purpose,bedrooms,date_added,agency,agent,Area Type,Area Size,Area Category
0,237062,3325,https://www.zameen.com/Property/g_10_g_10_2_gr...,Flat,10000000,G-10,Islamabad,Islamabad Capital,33.679890,73.012640,2,4 Marla,For Sale,2,02-04-2019,NaN,NaN,Marla,4.0,0-5 Marla
1,346905,3236,https://www.zameen.com/Property/e_11_2_service...,Flat,6900000,E-11,Islamabad,Islamabad Capital,33.700993,72.971492,3,5.6 Marla,For Sale,3,05-04-2019,NaN,NaN,Marla,5.6,5-10 Marla
2,386513,764,https://www.zameen.com/Property/islamabad_g_15...,House,16500000,G-15,Islamabad,Islamabad Capital,33.631486,72.926559,6,8 Marla,For Sale,5,07-17-2019,NaN,NaN,Marla,8.0,5-10 Marla
3,656161,340,https://www.zameen.com/Property/islamabad_bani...,House,43500000,Bani Gala,Islamabad,Islamabad Capital,33.707573,73.151199,4,2 Kanal,For Sale,4,04-05-2019,NaN,NaN,Kanal,2.0,1-5 Kanal
4,841645,3226,https://www.zameen.com/Property/dha_valley_dha...,House,7000000,DHA Defence,Islamabad,Islamabad Capital,33.492591,73.301339,3,8 Marla,For Sale,3,07-10-2019,Easy Property,Muhammad Junaid Ceo Muhammad Shahid Director,Marla,8.0,5-10 Marla


In [5]:
df['city'].nunique()

5

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168446 entries, 0 to 168445
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   property_id    168446 non-null  int64  
 1   location_id    168446 non-null  int64  
 2   page_url       168446 non-null  object 
 3   property_type  168446 non-null  object 
 4   price          168446 non-null  int64  
 5   location       168446 non-null  object 
 6   city           168446 non-null  object 
 7   province_name  168446 non-null  object 
 8   latitude       168446 non-null  float64
 9   longitude      168446 non-null  float64
 10  baths          168446 non-null  int64  
 11  area           168446 non-null  object 
 12  purpose        168446 non-null  object 
 13  bedrooms       168446 non-null  int64  
 14  date_added     168446 non-null  object 
 15  agency         124375 non-null  object 
 16  agent          124374 non-null  object 
 17  Area Type      168446 non-nul

In [7]:
# Properties with "0"-values
df.describe()

,property_id,location_id,price,latitude,longitude,baths,bedrooms,Area Size
count,1.684460e+05,168446.000000,1.684460e+05,168446.000000,168446.000000,168446.000000,168446.000000,168446.000000
mean,1.559626e+07,4375.936395,1.776576e+07,29.859519,71.239804,2.874227,3.179422,5.892188
std,2.251207e+06,3776.561581,3.531003e+07,3.807870,3.133042,2.463400,1.971401,5.778327
min,8.657500e+04,1.000000,0.000000e+00,11.052446,25.906027,0.000000,0.000000,0.000000
25%,1.488320e+07,1058.000000,1.750000e+05,24.948536,67.130363,0.000000,2.000000,3.000000
50%,1.665851e+07,3286.000000,8.500000e+06,31.459784,73.056182,3.000000,3.000000,5.000000
75%,1.708662e+07,7220.000000,1.950000e+07,33.560887,73.259870,4.000000,4.000000,8.000000
max,1.735772e+07,14220.000000,2.000000e+09,73.184088,80.161430,403.000000,68.000000,800.000000


## Values per property

In [8]:
df['location_id'].nunique()

4198

In [9]:
# Extra from documentaion: "Penthouse"
df['property_type'].value_counts()

property_type
House            105468
Flat              38238
Upper Portion     13774
Lower Portion      9229
Room                685
Farm House          657
Penthouse           395
Name: count, dtype: int64

In [10]:
df['location'].nunique()

1509

In [11]:
df['city'].value_counts()

city
Karachi       60484
Lahore        41427
Islamabad     37426
Rawalpindi    20976
Faisalabad     8133
Name: count, dtype: int64

In [12]:
df['province_name'].value_counts()

province_name
Punjab               70536
Sindh                60484
Islamabad Capital    37426
Name: count, dtype: int64

In [13]:
df['baths'].value_counts()

baths
0      42382
3      34603
2      25754
4      19407
6      16648
5      15303
7       5385
1       5356
8       2035
10       855
9        696
12        10
11         5
13         3
14         3
403        1
Name: count, dtype: int64

In [14]:
df['purpose'].value_counts()

purpose
For Sale    120655
For Rent     47791
Name: count, dtype: int64

In [15]:
df['bedrooms'].value_counts()

bedrooms
3     45867
2     30741
5     23452
4     23027
0     22704
6     11929
1      4434
7      2973
8      1451
9       782
10      611
11      416
12       25
14        8
15        7
16        4
28        4
13        3
25        3
18        2
21        1
68        1
27        1
Name: count, dtype: int64

## Empty values

In [16]:
df.isna().sum()

property_id          0
location_id          0
page_url             0
property_type        0
price                0
location             0
city                 0
province_name        0
latitude             0
longitude            0
baths                0
area                 0
purpose              0
bedrooms             0
date_added           0
agency           44071
agent            44072
Area Type            0
Area Size            0
Area Category        0
dtype: int64

## Outliers

In [17]:
df_select = df[df['price'] == 0]
df_select

,property_id,location_id,page_url,property_type,price,location,city,province_name,latitude,longitude,baths,area,purpose,bedrooms,date_added,agency,agent,Area Type,Area Size,Area Category
66537,16081186,8129,https://www.zameen.com/Property/rawalpindi_ran...,House,0,Range Road,Rawalpindi,Punjab,33.598593,73.018003,0,5 Marla,For Rent,0,05-06-2019,NaN,NaN,Marla,5.0,0-5 Marla
91608,16786134,12395,https://www.zameen.com/Property/karachi_gadap_...,Farm House,0,Gadap Road,Karachi,Sindh,25.053956,67.253365,0,8 Kanal,For Sale,0,06-21-2019,Bin Anus Real Estate,M. Humayun Jokhio,Kanal,8.0,5-10 Kanal
132855,17122202,1058,https://www.zameen.com/Property/rawalpindi_gul...,House,0,Gulshan Abad,Rawalpindi,Punjab,33.530556,73.061264,0,10 Marla,For Sale,0,07-05-2019,Al Nafai Investment Services,Jamil Hussain Shah,Marla,10.0,5-10 Marla


In [18]:
# Because we have few values, the rows can be deleted
len(df[df['price'] == 0]) / len(df) * 100

0.0017809861914203958

In [19]:
df.drop(df_select.index, inplace=True)
df.shape

(168443, 20)

In [20]:
# 0 and 403
df['baths'].value_counts()

baths
0      42379
3      34603
2      25754
4      19407
6      16648
5      15303
7       5385
1       5356
8       2035
10       855
9        696
12        10
11         5
13         3
14         3
403        1
Name: count, dtype: int64

In [21]:
df_select = df[df['baths'] >= 11]
df_select

,property_id,location_id,page_url,property_type,price,location,city,province_name,latitude,longitude,baths,area,purpose,bedrooms,date_added,agency,agent,Area Type,Area Size,Area Category
8791,10692266,8950,https://www.zameen.com/Property/lahore_shadman...,House,125000000,Shadman 2,Lahore,Punjab,31.535451,74.337407,11,2 Kanal,For Sale,11,07-03-2019,NaN,NaN,Kanal,2.0,1-5 Kanal
19947,12717357,329,https://www.zameen.com/Property/islamabad_e_11...,House,70000000,E-11,Islamabad,Islamabad Capital,33.698137,72.978215,11,1 Kanal,For Sale,11,06-19-2019,Sulman Real Estate & Builders,Hamza Rehman,Kanal,1.0,1-5 Kanal
89191,16751129,6924,https://www.zameen.com/Property/jamshed_town_j...,House,240000000,Jamshed Town,Karachi,Sindh,24.879379,67.045557,13,2 Kanal,For Sale,10,06-21-2019,Fine Estate,TARIQ MAHMOOD,Kanal,2.0,1-5 Kanal
89434,16754350,13726,https://www.zameen.com/Property/karachi_darul_...,House,180000000,Darul Aman Society,Karachi,Sindh,24.876054,67.067893,11,19 Marla,For Sale,9,06-21-2019,Mehran Real Estate,Mehran Real Estate,Marla,19.0,15-20 Marla
96053,16853359,3318,https://www.zameen.com/Property/f_8_f_8_markaz...,House,145000000,F-8,Islamabad,Islamabad Capital,33.711489,73.040457,12,1.3 Kanal,For Sale,9,06-26-2019,R. A Consultants,Abdul Raffay,Kanal,1.3,1-5 Kanal
111392,16997111,3282,https://www.zameen.com/Property/f_11_f_11_1_f_...,House,112500000,F-11,Islamabad,Islamabad Capital,33.679211,72.988787,12,1 Kanal,For Sale,9,06-28-2019,Pace Land Linkers,Jamil Ahmed Awan,Kanal,1.0,1-5 Kanal
112313,17004069,3239,https://www.zameen.com/Property/e_11_3_mpchs_i...,House,75000000,E-11,Islamabad,Islamabad Capital,33.703099,72.980032,12,1 Kanal,For Sale,10,06-29-2019,Team Work Construction & Real Estate,Muhammad Naeem,Kanal,1.0,1-5 Kanal
114794,17022707,418,https://www.zameen.com/Property/jamshed_town_s...,House,120000000,Jamshed Town,Karachi,Sindh,24.859299,67.055311,12,2 Kanal,For Sale,9,06-30-2019,Property World,Property World,Kanal,2.0,1-5 Kanal
123082,17069079,3154,https://www.zameen.com/Property/d_12_d_12_1_ho...,House,300000,D-12,Islamabad,Islamabad Capital,33.697565,72.944241,13,1.2 Kanal,For Rent,10,07-02-2019,Pak Land Property,Mardan obaid,Kanal,1.2,1-5 Kanal
129605,17101832,587,https://www.zameen.com/Property/rawalpindi_kha...,House,15000000,Khayaban-e-Sir Syed,Rawalpindi,Punjab,33.638847,73.048089,403,5 Marla,For Sale,6,07-04-2019,Al-Rehman Estate & Developers,Raja Ahsan Rehman H,Marla,5.0,0-5 Marla


In [22]:
df.drop(df[df['baths'] == 403].index, inplace=True)
df.shape

(168442, 20)

In [23]:
# Lots of "0"
df['bedrooms'].value_counts()

bedrooms
3     45867
2     30741
5     23452
4     23027
0     22701
6     11928
1      4434
7      2973
8      1451
9       782
10      611
11      416
12       25
14        8
15        7
16        4
28        4
13        3
25        3
18        2
21        1
68        1
27        1
Name: count, dtype: int64

In [24]:
percent = round(len(df[df['bedrooms'] == 0]) / len(df) * 100, 2)
print('{}%'.format(percent))

13.48%


In [25]:
df['bedrooms'] = df['bedrooms'].replace(0, df['bedrooms'].median())
df['bedrooms'].median()

3.0

In [26]:
df[df['bedrooms'] >= 20][['price', 'bedrooms']].sort_values('price')

,price,bedrooms
130132,300000,21
144418,550000,25
152009,900000,25
120075,1300000,28
139738,1300000,28
164716,1300000,28
164717,1300000,28
140891,2500000,68
144837,80000000,27
129557,960000000,25


In [27]:
df.drop(df[df['bedrooms'] == 68].index, inplace=True)
df.shape

(168441, 20)

In [28]:
df['bedrooms'].value_counts()

bedrooms
3     68568
2     30741
5     23452
4     23027
6     11928
1      4434
7      2973
8      1451
9       782
10      611
11      416
12       25
14        8
15        7
16        4
28        4
13        3
25        3
18        2
21        1
27        1
Name: count, dtype: int64

In [29]:
df['Area Size'].value_counts()

Area Size
1.0      20286
5.0      20049
10.0     17450
8.0      10216
4.0       7259
         ...  
18.9         1
46.0         1
600.0        1
26.0         1
800.0        1
Name: count, Length: 247, dtype: int64

In [30]:
# Because we have few values, the rows can be deleted
df_select = df[df['Area Size'] == 0]
len(df_select) / len(df) * 100

0.007124156232746184

In [31]:
df.drop(df_select.index, inplace=True)
df.shape

(168429, 20)

In [32]:
df[df['Area Size'] >= 500][['price', 'Area Size']].sort_values('Area Size')

,price,Area Size
30276,35000,500.0
125631,300000,500.0
167017,115000,500.0
143744,70000,500.0
168178,70000,500.0
68989,2200000,600.0
126842,13000,650.0
165150,16000,800.0


## Q&A

* Kaggle dataset differences: basic, EDA, model
* Row numbers for previous datasets
* What columns to keep?
* Is there any need to remove other rows?